In [ ]:
"""
Code to predict hate speech label by using explanations from T5.
"""

In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 99.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 101.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 100.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.7/211.7 kB 28.1 MB/s eta 0:00:

In [2]:
import pandas as pd
from simpletransformers.t5 import T5Model

In [3]:
## Read data
train_df_base = pd.read_csv("/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/df_train.csv").astype(str)[['prefix','input_text','target_text']]
eval_df_base = pd.read_csv("/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/df_val.csv").astype(str)[['prefix','input_text','target_text']]
test_df_base = pd.read_csv("/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/df_test.csv").astype(str)[['prefix','input_text','target_text']]

## Select data-points with prefix = 'explanation'
train_df_exp = train_df_base[train_df_base['prefix']=='explanation'].copy()
eval_df_exp = eval_df_base[eval_df_base['prefix']=='explanation'].copy()
test_df_exp = test_df_base[test_df_base['prefix']=='explanation'].copy()

train_df_exp.reset_index(drop=True, inplace=True)
eval_df_exp.reset_index(drop=True, inplace=True)
test_df_exp.reset_index(drop=True, inplace=True)

print("Train shape: ", train_df_exp.shape)
print("Val shape: ", eval_df_exp.shape)
print("Test shape: ", test_df_exp.shape)

Train shape:  (14057, 3)
Val shape:  (1786, 3)
Test shape:  (1759, 3)


In [20]:
## Output

train_explanations_path =  "/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/t5-for-explanation/df_train_pred_exp.csv"
val_explanations_path =  "/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/t5-for-explanation/df_val_pred_exp.csv"
test_explanations_path =  "/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/t5-for-explanation/df_test_pred_exp.csv"

model_op_path = "/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/t5_ip_sent_exp"

In [5]:
## Get predictions from T5-for-explanation
model_args = {
    "overwrite_output_dir": True,
    "max_seq_length": 250,
    "eval_batch_size": 32,
    "use_multiprocessing": False,
    "num_beams": 3,
    "do_sample": True,
    "max_length": 50,
    "top_k": 50,
    "top_p": 0.95,
    "num_return_sequences": 3,
}

# Load the trained model
model = T5Model("t5", "/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/t5-for-explanation/outputs/best_model", args=model_args)


In [6]:
# Predict explanations function

def predict_explanations(df):
  # Prepare the data for predictions
  to_predict = [
      prefix + ": " + str(input_text)
      for prefix, input_text in zip(df["prefix"].tolist(), df["input_text"].tolist())
  ]
  truth = df["target_text"].tolist()
  tasks = df["prefix"].tolist()

  # Get the model predictions
  preds = model.predict(to_predict)

  ## Taking only the top (first) predictions
  top_preds = [pred[0] for pred in preds]
  df["predicted"] = top_preds
  df['all_preds'] = preds



In [7]:
## Predict on train, val and test -  chance of overfitting

## Uncomment the following for predictions
# print("Predicting train")
# predict_explanations(train_df_exp)
# train_df_exp.to_csv(train_explanations_path, index=False)

# print("Predicting val")
# predict_explanations(eval_df_exp)
# eval_df_exp.to_csv(val_explanations_path, index=False)

# print("Predicting test")
# predict_explanations(test_df_exp)
# test_df_exp.to_csv(test_explanations_path, index=False)


train_df_exp = pd.read_csv(train_explanations_path)
eval_df_exp = pd.read_csv(val_explanations_path)
test_df_exp = pd.read_csv(test_explanations_path)

In [8]:
train_df_exp.head()


,prefix,input_text,target_text,predicted,all_preds
0,explanation,freak hoe will forever snap,this sentence uses derogatory language toward...,this sentence uses derogatory language towards...,['this sentence uses derogatory language towar...
1,explanation,i wonder if they had to walk out in schools ne...,"the use of the term ""ghetto"" could potentiall...",this sentence contains derogatory language tow...,['this sentence contains derogatory language t...
2,explanation,someone took my fucking dyke pin i am going to...,"the use of the term ""dyke"" as an insult towar...",this sentence uses derogatory language towards...,['this sentence uses derogatory language towar...
3,explanation,i am done with you i am not helping you get yo...,this statement uses hostile language directed...,this sentence uses derogatory language towards...,['this sentence uses derogatory language towar...
4,explanation,mr trudeau you pledged 8.760523525193214 milli...,this statement raises important concerns abou...,this sentence contains derogatory language tow...,['this sentence contains derogatory language t...


In [10]:
## Extracting labels

train_df_label = train_df_base[train_df_base['prefix']=='label'].copy()
eval_df_label = eval_df_base[eval_df_base['prefix']=='label'].copy()
test_df_label = test_df_base[test_df_base['prefix']=='label'].copy()

train_df_label.reset_index(drop=True, inplace=True)
eval_df_label.reset_index(drop=True, inplace=True)
test_df_label.reset_index(drop=True, inplace=True)

print("Train shape: ", train_df_label.shape)
print("Eval shape: ", eval_df_label.shape)
print("Test shape: ", test_df_label.shape)

Train shape:  (14072, 3)
Eval shape:  (1787, 3)
Test shape:  (1761, 3)


In [11]:
train_df_label.head()

,prefix,input_text,target_text
0,label,ok gonna block annoying faggots,offensive
1,label,Michael Michael how about 34.739257035498376% ...,normal
2,label,fucking yid everytime,hate
3,label,im so horny looking for someone who can help m...,normal
4,label,Benjamin well that what kennedy said to get he...,normal


In [18]:
# Function to get modeling data
def get_model_data(df_label, df_exp):
  df = df_label.merge(df_exp[['input_text','predicted']], on = ['input_text'], how = 'inner')
  df.rename(columns={'input_text':'sentence', 'predicted':'predicted_exp'}, inplace=True)
  df['input_text'] = df.apply(lambda row: row['sentence'] + ' ' + row['predicted_exp'], axis=1)
  df = df[['prefix','input_text','target_text']].copy()
  return df

train_df = get_model_data(train_df_label, train_df_exp)
eval_df = get_model_data(eval_df_label, eval_df_exp)
test_df = get_model_data(test_df_label, test_df_exp)

print("Train shape: ", train_df.shape)
print("Val shape: ", eval_df.shape)
print("Test shape: ", test_df.shape)

Train shape:  (14071, 3)
Val shape:  (1786, 3)
Test shape:  (1759, 3)


In [23]:
## Store data
train_df.to_csv(model_op_path + "/train_df.csv", index=False)
eval_df.to_csv(model_op_path + "/eval_df.csv", index=False)
test_df.to_csv(model_op_path + "/test_df.csv", index=False)

In [25]:
model_args_label = {
    "max_seq_length": 250,
    "train_batch_size": 16,
    "eval_batch_size": 16,
    "num_train_epochs": 5,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 15000,
    "evaluate_during_training_verbose": True,
    "use_multiprocessing": False,
    "fp16": False,
    "save_steps": -1,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "learning_rate":1e-4,
    "weight_decay":0.01
}

model = T5Model("t5", "t5-base", args=model_args_label)
model.train_model(train_df, eval_data=eval_df, output_dir=model_op_path)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


  0%|          | 0/14071 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/880 [00:00<?, ?it/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

Running Epoch 4 of 5:   0%|          | 0/880 [00:00<?, ?it/s]

  0%|          | 0/1786 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

(4400,
 {'global_step': [880, 1760, 2640, 3520, 4400],
  'eval_loss': [0.4105555094512446,
   0.35880826759551254,
   0.38747751087482485,
   0.47175043668331845,
   0.44663812898631605],
  'train_loss': [0.599237322807312,
   0.44238632917404175,
   0.27089691162109375,
   0.2961962819099426,
   0.22873736917972565]})

In [26]:
model_args_test = {
    "overwrite_output_dir": True,
    "max_seq_length": 250,
    "eval_batch_size": 32,
    "use_multiprocessing": False,
    "num_beams": 3,
    "do_sample": True,
    "max_length": 50,
    "top_k": 50,
    "top_p": 0.95,
    "num_return_sequences": 3,
}

# Load the trained model
model = T5Model("t5", "/content/outputs/best_model", args=model_args_test)


In [27]:
# Prepare the data for testing
to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(test_df["prefix"].tolist(), test_df["input_text"].tolist())
]
truth = test_df["target_text"].tolist()
tasks = test_df["prefix"].tolist()

In [28]:
# Get the model predictions
preds = model.predict(to_predict)

## Taking only the top (first) predictions
top_preds = [pred[0] for pred in preds]
test_df["predicted"] = top_preds
test_df['all_preds'] = preds

## Saving predictions
test_df.to_csv(model_op_path + '/df_test_predictions.csv', index=False)


Generating outputs:   0%|          | 0/55 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3745: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/5277 [00:00<?, ?it/s]

In [32]:
acc = (test_df['predicted']==test_df['target_text']).sum()/test_df.shape[0]


0.6969869243888573

In [34]:
!mv '/content/outputs' '/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/t5_ip_sent_exp/'